# ASL Recognition with MobileNetV2

This notebook trains a MobileNetV2 model to recognize American Sign Language (ASL) alphabets (A-Z).
It uses the dataset split into `data/train`, `data/test`, and `data/val`.

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

## Data Preparation
We use `ImageDataGenerator` to load and preprocess images. MobileNetV2 expects pixel values in [-1, 1], so we use `preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input`.

In [2]:
IMG_SIZE = 224
BATCH_SIZE = 32
DATA_DIR = 'data'

train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    os.path.join(DATA_DIR, 'train'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    os.path.join(DATA_DIR, 'val'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    os.path.join(DATA_DIR, 'test'),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 22704 images belonging to 26 classes.
Found 6558 images belonging to 26 classes.
Found 6579 images belonging to 26 classes.


## Model Construction
We use MobileNetV2 as the base model, pre-trained on ImageNet. We freeze the base layers and add a custom classification head.

In [3]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze base model layers
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(26, activation='softmax')(x)  # 26 classes for A-Z

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

## Training
We train the model with EarlyStopping to prevent overfitting.

In [4]:
epochs = 20

callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('asl_model.h5', monitor='val_accuracy', save_best_only=True)
]

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/20
710/710 [==============================] - 461s 638ms/step - loss: 1.9317 - accuracy: 0.4629 - val_loss: 0.5942 - val_accuracy: 0.9565
Epoch 2/20
710/710 [==============================] - 254s 358ms/step - loss: 0.7789 - accuracy: 0.7974 - val_loss: 0.1979 - val_accuracy: 0.9890
Epoch 3/20
710/710 [==============================] - 182s 256ms/step - loss: 0.4480 - accuracy: 0.8869 - val_loss: 0.0959 - val_accuracy: 0.9976
Epoch 4/20
710/710 [==============================] - 138s 194ms/step - loss: 0.3027 - accuracy: 0.9272 - val_loss: 0.0587 - val_accuracy: 0.9976
Epoch 5/20
710/710 [==============================] - 137s 192ms/step - loss: 0.2303 - accuracy: 0.9467 - val_loss: 0.0424 - val_accuracy: 0.9971
Epoch 6/20
710/710 [==============================] - 169s 237ms/step - loss: 0.1824 - accuracy: 0.9588 - val_loss: 0.0288 - val_accuracy: 0.9983
Epoch 7/20
710/710 [==============================] - 175s 246ms/step - loss: 0.1432 - accuracy: 0.9676 - val_loss: 0.0181 -

## Fine-tuning (Optional)
Unfreeze the top layers of the base model and train with a lower learning rate to improve accuracy.

In [5]:
# Unfreeze the top 20 layers
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

history_finetune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=callbacks
)

Epoch 1/10
710/710 [==============================] - 141s 195ms/step - loss: 0.2513 - accuracy: 0.9246 - val_loss: 6.0173e-04 - val_accuracy: 1.0000
Epoch 2/10
710/710 [==============================] - 136s 192ms/step - loss: 0.0589 - accuracy: 0.9833 - val_loss: 8.1069e-04 - val_accuracy: 0.9997
Epoch 3/10
710/710 [==============================] - 210s 296ms/step - loss: 0.0348 - accuracy: 0.9912 - val_loss: 8.1706e-04 - val_accuracy: 0.9998
Epoch 4/10
710/710 [==============================] - 151s 213ms/step - loss: 0.0245 - accuracy: 0.9930 - val_loss: 7.1872e-04 - val_accuracy: 0.9998
Epoch 5/10
710/710 [==============================] - 137s 193ms/step - loss: 0.0195 - accuracy: 0.9952 - val_loss: 6.9741e-04 - val_accuracy: 0.9998
Epoch 6/10
710/710 [==============================] - 135s 190ms/step - loss: 0.0135 - accuracy: 0.9968 - val_loss: 7.1365e-04 - val_accuracy: 0.9998


## Evaluation
Evaluate the model on the test set.

In [6]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc * 100:.2f}%')

206/206 [==============================] - 59s 289ms/step - loss: 9.8047e-04 - accuracy: 0.9997
Test Accuracy: 99.97%


In [7]:
# Save the final model
model.save('asl_model_final.h5')